Search Data Processing
=========================



In [ ]:
import pandas as pd
import re

In [ ]:
search_data = pd.read_csv("./data/sample_openData_searchTerms_clean.csv")

In [ ]:
search_data.shape

In [ ]:
search_data.tail()

In [ ]:
#search_data[search_data['Total Unique Searches']<5]

In [ ]:
search_terms = list(set(search_data['Search Term']))

In [ ]:
len(search_terms)

In [ ]:
print search_terms

In [ ]:
search_data.ix[search_data["Search Term"] == "194415"]

In [ ]:
search_terms_data = search_data[["Search Term"]]
search_terms_data = search_terms_data.rename(columns={"Search Term": "search_term"})

In [ ]:
search_terms_data.head()

In [ ]:
search_terms_data['processed_data'] = search_terms_data.search_term\
                                        .apply(lambda text: text.decode('utf-8','ignore').encode("utf-8"))\
                                        .apply(lambda text: text.lower())

In [ ]:
search_terms_list =  list(set(search_terms_data.processed_data))
print search_terms_list

#### We need to generally tag searches in categories of quality

Many search terms indicate that users are not informed on what the purpose of the website is or how to properly use search

Search Tags
* Good Quality Search
    * complete words or phrases
    * minor typos
* Bad Quality
    * Addresses
    * Dates
    * Zipcodes or just a string of numbers
    * General nonsenses eg. ('></script><script>alert(1)</script>', '///', '16exc-3031') 

In [ ]:
dates_r = re.compile('[0-9]/[0-9]/[0-9]')
numbers_r = re.compile('^[0-9][0-9]*[0-9]$')
html_r = re.compile('^<.*>$')


In [ ]:
filter(dates_r.match, search_terms_list)

In [ ]:
print filter(html_r.match, search_terms_list)